## For Next Time

- Add a date column for the regression.
- Goal is to essentially have poll company + date = next_poll spread 
- Look at white papers to see schedule for date. 
- Fit the model everyday, generate a prediction for every poll, and see if it comes out the next day. 
- Real-time predictions with errors. 
- Just do Joe-Biden Trump, print out predictions for most common polls; maybe make a .txt file output and then compare. 
- Real Clear Polling for that as the data source 
- Sentiment Analysis + Trending search terms
- Plot polling aggregation by day accounting for some sort of bias by the polling companies. 
- Use some sort of standard time series model using all of the polling data. 
- Gaussian process, GAM, splines, time series, polynomial regression, for a more flexible model. 

Poll_outcome = poll company + time_since +....?

Keep thinking about google trends somehow

In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px

# Suppress a specific warning
warnings.simplefilter("ignore", category=FutureWarning)

In [68]:
base_url = "https://www.realclearpolling.com/polls/"
sample_url = "president/general/2024/trump-vs-biden"
url = base_url + sample_url

In [69]:
# Create a webdriver instance and get the page source
driver = webdriver.Chrome()
driver.get(url)

# Allow time for dynamic content to load (you may need to adjust the sleep duration)
time.sleep(3)

# Get the page source after dynamic content has loaded
html_content = driver.page_source

# Close the webdriver
driver.quit()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html5lib')

# Now you can extract the table data using the same approach as before
table = soup.find_all('table')

if len(table)==2:
    table = table[1]
else:
    table = table[0]

table_data = []
for row in table.find_all('tr'):
    row_data = [cell.text.strip() for cell in row.find_all(['td','th'])]
    table_data.append(row_data)

In [79]:
#Convert table data to a dataframe
#We remove the first row since this is RCP summary data that we don't want to use

current_year = str(datetime.now().year)
prev_year = str(datetime.now().year-1)


df = pd.DataFrame(table_data[2:], columns=table_data[0])
df["Difference"] = df["Trump (R)"].astype(float) - df["Biden (D)"].astype(float)
df["Type of Voter"] = df["sample"].str.split(" ").str[1]
df["Sample Size"] = pd.to_numeric(df["sample"].str.split(" ").str[0], errors="coerce").fillna(0).astype(int)
#We need to add the year to the date to make it a datetime object
#We need to make sure the year we add is the year the poll was taken, not necessarily the current year
df["End Date"] = df["date"].str.split("-").str[1] 
df["Poll Month"] = df["date"].str.split("-").str[1].str.split("/").str[0]
df["Poll Month"] = df["Poll Month"].astype(int)
first_dec = df[df["Poll Month"]==12].index[0]
df["Year"] = [current_year]*first_dec + [prev_year]*(len(df)-first_dec)
df["End Date"] = df["End Date"] + "/" + df["Year"]
df["End Date"] = np.array(pd.to_datetime(df["End Date"], format="mixed"))
df = df[df["Type of Voter"].isin(["RV", "LV"])]
df.head()

,pollster,date,sample,moe,Trump (R),Biden (D),spread,Difference,Type of Voter,Sample Size,End Date,Poll Month,Year
0,Emerson,2/13 - 2/14,1225 RV,2.7,45,44,Trump+1,1.0,RV,1225,2024-02-14,2,2024
1,Economist/YouGov,2/11 - 2/13,1470 RV,3.1,44,44,Tie,0.0,RV,1470,2024-02-13,2,2024
2,Morning Consult,2/9 - 2/11,6164 RV,1.0,43,42,Trump+1,1.0,RV,6164,2024-02-11,2,2024
3,Economist/YouGov,2/4 - 2/6,1399 RV,3.0,44,43,Trump+1,1.0,RV,1399,2024-02-06,2,2024
4,Morning Consult,2/4 - 2/6,6138 RV,1.0,45,40,Trump+5,5.0,RV,6138,2024-02-06,2,2024


In [80]:
#Create a time series plot of the candidates' polling numbers using plotly
fig = px.scatter(df, x="End Date", y=["Trump (R)", "Biden (D)"], title="Trump vs. Biden Polling Numbers")
fig.show()

In [81]:
df["Days Since 01-01-23"] = (df["End Date"] - pd.to_datetime("01-01-23")).dt.days

df.head()

,pollster,date,sample,moe,Trump (R),Biden (D),spread,Difference,Type of Voter,Sample Size,End Date,Poll Month,Year,Days Since 01-01-23
0,Emerson,2/13 - 2/14,1225 RV,2.7,45,44,Trump+1,1.0,RV,1225,2024-02-14,2,2024,409
1,Economist/YouGov,2/11 - 2/13,1470 RV,3.1,44,44,Tie,0.0,RV,1470,2024-02-13,2,2024,408
2,Morning Consult,2/9 - 2/11,6164 RV,1.0,43,42,Trump+1,1.0,RV,6164,2024-02-11,2,2024,406
3,Economist/YouGov,2/4 - 2/6,1399 RV,3.0,44,43,Trump+1,1.0,RV,1399,2024-02-06,2,2024,401
4,Morning Consult,2/4 - 2/6,6138 RV,1.0,45,40,Trump+5,5.0,RV,6138,2024-02-06,2,2024,401


In [89]:
ndf = pd.get_dummies(df[["pollster"]], drop_first=True)
ndf["Days Since 01-01-23"] = df["Days Since 01-01-23"]
ndf.head()

,pollster_CBS News,pollster_CNN,pollster_Daily Kos/Civiqs,pollster_Data for Progress (D)**,pollster_Economist/YouGov,pollster_Emerson,pollster_FOX News,pollster_Federalist/Susquehanna,pollster_Grinnell/Selzer,pollster_Harvard-Harris,pollster_I&I/TIPP,pollster_Insider Advantage,pollster_InsiderAdvantage,pollster_Marist,pollster_Marquette,pollster_Morning Consult,pollster_NBC News,pollster_NPR/PBS/Marist,pollster_NY Times/Siena,pollster_New York Post,pollster_PPP (D),pollster_Politico/Morning Consult,pollster_Quinnipiac,pollster_Rasmussen Reports,pollster_Reuters/Ipsos,pollster_SurveyUSA,pollster_Susquehanna,pollster_The Messenger/HarrisX,pollster_Trafalgar Group (R),pollster_USA Today/Suffolk,pollster_Wall Street Journal,pollster_Yahoo News,pollster_Yahoo News**,Days Since 01-01-23
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,409
1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,408
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,406
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,401
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,401


In [109]:
# Build a categorical regression model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = ndf
y = df[["Difference"]]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = LinearRegression()
model.fit(X, y)


LinearRegression()

In [110]:
from datetime import datetime

today_date = datetime.now().strftime("%m-%d-%Y") #Get today's date
days_since_2023 = (pd.to_datetime(today_date) - pd.to_datetime("01-01-2023")).days
cols = ndf.columns[:-1]
diag = np.diag(np.ones(len(cols)))

tomorrow_df = pd.DataFrame(diag, columns=cols)
tomorrow_df["Days Since 01-01-23"] = days_since_2023 + 1 #Add 1 to the days since 2023 to make it the next day
tomorrow_df.head()

tomorrow_pred = model.predict(tomorrow_df)




In [111]:
#Format the prediction into a readable text

#Make a dataframe 
tomorrow_pred_df = pd.DataFrame(tomorrow_pred, columns=["Predicted Difference"])
tomorrow_pred_df.set_index(cols, inplace=True)
tomorrow_pred_df

,Predicted Difference
pollster_CBS News,2.504714
pollster_CNN,3.499684
pollster_Daily Kos/Civiqs,0.095577
pollster_Data for Progress (D)**,-1.743451
pollster_Economist/YouGov,0.444247
pollster_Emerson,2.224215
pollster_FOX News,1.846906
pollster_Federalist/Susquehanna,-1.656891
pollster_Grinnell/Selzer,0.628796
pollster_Harvard-Harris,5.355896


In [ ]:
# Make an excel file where each sheet is that day's prediction

tomorrow_pred_df.to_excel("Datasets/Predictions.xlsx",sheet_name=today_date)
